# Basic usage of ABCD database

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from ase.io import iread, read
from utils.ext_xyz import XYZReader

from abcd import ABCD

First of all, we need to define the url of the database. It could be local or remote:

- direct access: url = 'mongodb://localhost:27017'
- api access: url = 'http://localhost/api'

using with statement to catch the riased exceptions. You may can ignore them  but in that case need to handle all the unexpected events. (cannot connect to db, lost connection, wrong filter, wrong url, etc. )

In [12]:
abcd.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [13]:
abcd.db


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'abcd')

In [16]:
url = 'mongodb://localhost:27017/'
abcd =  ABCD(url, db_name='test')

print(abcd)

MongoDatabase(url=localhost:27017, db=test, collection=atoms)


In [21]:
abcd.print_info()

================== ABCD MongoDB ==================
      type: mongodb
      host: localhost
      port: 27017
        db: test
collection: atoms
number of confs: 14


## Cleanup 

WARNING!! Remove all elements from the database.
Only supported in the case of local access

In [22]:
with abcd as db:
    db.destroy()

## Uploading configurations

In [46]:


direcotry = Path('data/')
file = direcotry / 'bcc_bulk_54_expanded_2_high.xyz'
# file = direcotry / 'GAP_1.xyz'

Uploading configurations on-by-one directly from an ase atoms object:

In [47]:
%%time
with abcd as db:

    for atoms in iread(file.as_posix(), index=slice(None)):
        
        # Hack to fix the representation of forces
        
        atoms.calc.results['forces'] = atoms.arrays['force']
        del(atoms.arrays['force'])
            
        db.push(atoms)
    

CPU times: user 29.9 ms, sys: 2.09 ms, total: 32 ms
Wall time: 38.9 ms


Reading the trajectory from file:

In [48]:
%%time
traj = read(file.as_posix(), index=slice(None))
len(traj)

CPU times: user 20.2 ms, sys: 3 ms, total: 23.2 ms
Wall time: 23.8 ms


In [49]:
%%time
with XYZReader(file) as reader:
    traj = list(reader.read_atoms(forces_label='force'))


CPU times: user 8.88 ms, sys: 1.52 ms, total: 10.4 ms
Wall time: 9.74 ms


Pushing the whole trajectory to the database:

In [50]:
%%time
db.push(traj)

CPU times: user 8.18 ms, sys: 788 µs, total: 8.97 ms
Wall time: 10.2 ms


Uploading a whole file and injecting to the database on the server side:

In [51]:
%%time
with abcd as db:
    db.upload(file.as_posix())

CPU times: user 24.3 ms, sys: 4.64 ms, total: 28.9 ms
Wall time: 28.9 ms


An alternative way to upload file to database:

In [52]:
%%time
with abcd as db, XYZReader(file) as reader:
    db.push(reader.read_atoms(forces_label='force'))

CPU times: user 18.7 ms, sys: 2.04 ms, total: 20.7 ms
Wall time: 22.5 ms


In [53]:
abcd.info()

{'host': 'localhost',
 'port': 27017,
 'db': 'test',
 'collection': 'atoms',
 'number of confs': 224}

## Uploading


In [ ]:
for file in Path('data/').glob('*.xyz'):
    print(file)
#     with abcd as db, XYZReader(file) as reader:
#         db.push(reader.read_atoms())
        
    traj = read(file.as_posix(), index=slice(None))
    db.push(traj)

In [ ]:
for file in Path('GB_alphaFe_001/tilt/').glob('*.xyz'):
    print(file)
    gb_params = {
        'name': 'alphaFe',
        'type': 'tilt',
        'params': file.name[:-4]
        
    }    

    traj = read(file.as_posix(), index=slice(None))
    db.push(traj, extra_info={'GB_params': gb_params})

In [ ]:
#     with abcd as db, XYZReader(file) as reader:
#         db.push(reader.read_atoms(), extra_info={'GB_params': gb_params})
